In [0]:
"""
File setup
"""
# from google.colab import drive
# drive.mount("/content/drive/")
# import sys
# sys.path.append('/content/drive/My Drive/Colab Notebooks/Proj4/')
# import os
# os.chdir('/content/drive/My Drive/Colab Notebooks/Proj4/')

# !pip install -r ./requirements.txt

In [0]:
"""
Import packages
"""
import time

import cv2
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import png
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from colormap.colors import Color, hex2rgb
from sklearn.metrics import average_precision_score as ap_score
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from tqdm import tqdm
import tensorflow as tf

from dataset import FacadeDataset

In [0]:
def save_label(label, path):
    '''
    Function for ploting labels.
    '''
    colormap = [
        '#000000',
        '#0080FF',
        '#80FF80',
        '#FF8000',
        '#FF0000',
    ]
    assert(np.max(label)<len(colormap))
    colors = [hex2rgb(color, normalise=False) for color in colormap]
    w = png.Writer(label.shape[1], label.shape[0], palette=colors, bitdepth=4)
    with open(path, 'wb') as f:
        w.write(f, label)

def train(trainloader, net, criterion, optimizer, device, epoch):
    '''
    Function for training.
    '''
    start = time.time()
    running_loss = 0.0
    net = net.train()
    for images, labels in tqdm(trainloader, position=0, leave=True):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss = loss.item()
    end = time.time()
    print('[epoch %d] loss: %.3f elapsed time %.3f' %
          (epoch, running_loss, end-start))

def test(testloader, net, criterion, device):
    '''
    Function for testing.
    '''
    losses = 0.
    cnt = 0
    with torch.no_grad():
        net = net.eval()
        for images, labels in tqdm(testloader, position=0, leave=True):
            images = images.to(device)
            labels = labels.to(device)
            output = net(images)
            loss = criterion(output, labels)
            losses += loss.item()
            cnt += 1
    print(losses / cnt)
    return (losses/cnt)


def cal_AP(testloader, net, criterion, device):
    '''
    Calculate Average Precision
    '''
    losses = 0.
    cnt = 0
    with torch.no_grad():
        net = net.eval()
        preds = [[] for _ in range(5)]
        heatmaps = [[] for _ in range(5)]
        for images, labels in tqdm(testloader, position=0, leave=True):
            images = images.to(device)
            labels = labels.to(device)
            output = net(images).cpu().numpy()
            for c in range(5):
                preds[c].append(output[:, c].reshape(-1))
                heatmaps[c].append(labels[:, c].cpu().numpy().reshape(-1))

        aps = []
        for c in range(5):
            preds[c] = np.concatenate(preds[c])
            heatmaps[c] = np.concatenate(heatmaps[c])
            if heatmaps[c].max() == 0:
                ap = float('nan')
            else:
                ap = ap_score(heatmaps[c], preds[c])
                aps.append(ap)
            print("AP = {}".format(ap))
    return None


def get_result(testloader, net, device, folder='output_train'):
    result = []
    cnt = 1
    with torch.no_grad():
        net = net.eval()
        cnt = 0
        for images, labels in tqdm(testloader, position=0, leave=True):
            images = images.to(device)
            labels = labels.to(device)
            output = net(images)[0].cpu().numpy()
            c, h, w = output.shape
            assert(c == N_CLASS)
            y = np.zeros((h,w)).astype('uint8')
            for i in range(N_CLASS):
                mask = output[i]>0.7
                y[mask] = i
            gt = labels.cpu().data.numpy().squeeze(0).astype('uint8')
            save_label(y, './{}/y{}.png'.format(folder, cnt))
            save_label(gt, './{}/gt{}.png'.format(folder, cnt))
            plt.imsave(
                './{}/x{}.png'.format(folder, cnt),
                ((images[0].cpu().data.numpy()+1)*128).astype(np.uint8).transpose(1,2,0))

            cnt += 1

In [0]:
"""
Neural network hyperparameters and layers
"""
N_CLASS = 5
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.n_class = N_CLASS
        self.layers = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32, momentum=0.01),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64, momentum=0.01),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128, momentum=0.01),
            nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1),
            nn.BatchNorm2d(64, momentum=0.01),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1),
            nn.BatchNorm2d(32, momentum=0.01),
            nn.Conv2d(32, 5, 3, padding=1),
            nn.BatchNorm2d(5, momentum=0.01),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [0]:
"""
Set up datasets: TRAINING, EVAL, TEST
"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_data = FacadeDataset(flag='train', data_range=(0,800), onehot=False)
train_loader = DataLoader(train_data, batch_size=4)
test_data = FacadeDataset(flag='test_dev', data_range=(0,114), onehot=False)
test_loader = DataLoader(test_data, batch_size=1)
evaluation_data = FacadeDataset(flag='train', data_range=(800,906), onehot=False)
evaluation_loader = DataLoader(evaluation_data, batch_size=1)
ap_data = FacadeDataset(flag='test_dev', data_range=(0,114), onehot=True)
ap_loader = DataLoader(ap_data, batch_size=4)

In [0]:
name = 'retry_net'
net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), 1e-3, weight_decay=1e-5)
training_loss = np.zeros(100)
validation_loss = np.zeros(100)
print('\nStart training')
for epoch in range(0, 30):
    print('-----------------Epoch = %d-----------------' % (epoch+1))
    train(train_loader, net, criterion, optimizer, device, epoch+1)
    training_loss[epoch] = test(train_loader, net, criterion, device)
    validation_loss[epoch] = test(evaluation_loader, net, criterion, device)

print('\nFinished Training, Testing on test set')
test(test_loader, net, criterion, device)

In [0]:
print('\nGenerating Unlabeled Result')
result = get_result(test_loader, net, device, folder='output_test')
torch.save(net.state_dict(), './models/model_{}.pth'.format(name))
cal_AP(ap_loader, net, criterion, device)

In [0]:
epochs = np.linspace(1, 30, 30)
fig = plt.figure(figsize=(10,5))
plt.plot(epochs, training_loss[:30], label="Training")
plt.plot(epochs, validation_loss[:30], label="Validation")
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Average Loss")
fig.legend()
plt.show()

In [0]:
img = plt.imread("./london.JPG")
img = np.transpose(img, (2, 0, 1))
img = np.reshape(img, (1, 3, 256, 256))
img = torch.FloatTensor(img).to(device)
with torch.no_grad():
  output = net(img)[0].cpu().numpy()
  c, h, w = output.shape
  y = np.zeros((h,w)).astype('uint8')
  for i in range(N_CLASS):
      mask = output[i]>0.7
      y[mask] = i
  save_label(y, './london.png')